# Rasters to Vectors: Roads

This notebook demonstrates the process of converting an output of the Analytics Feed (road footprint raster) into a vector dataset, using the following steps:

1. Obtaining An Analytics Raster
2. Downloading Quad Raster
3. Visualizing Road Image
4. Converting Roads Raster to Vector Features using the following techniques:
    * GDAL CLI
    * Rasterio (no processing)
    * Rasterio (with simplification)
    * Rasterio (flexible function, filtering and simplification as example)

In [ ]:
import json
import os
import requests
import fiona
import matplotlib.pyplot as plt
from planet import Auth
from planet import Session, DataClient, OrdersClient
import rasterio
from rasterio import features as rfeatures
from rasterio.enums import Resampling
from rasterio.plot import show
import shapely
from shapely.geometry import shape as sshape

In [ ]:
# if your Planet API Key is not set as an environment variable, you can paste it below
API_KEY = os.environ.get('PL_API_KEY', 'PASTE_API_KEY_HERE')

client = Auth.from_key(API_KEY)

# Use our API key as the basic authentication username
apiAuth = (API_KEY, '')

## Obtain Analytics Raster

### Identify road feed feature for download

We want to download the most recent feature from the feed for road detection in Kirazli, Turkey.

In [ ]:
# This ID is for a subscription for monthly road detection in Kirazli, Turkey
subscription_id = 'f184516c-b948-406f-b257-deaa66c3f38a'

In [ ]:
# Planet Analytics API base url
PAA_BASE_URL = "https://api.planet.com/analytics/"

# Define collections endpoint
collections_endpoint = 'collections/'

# Construct the URL for the HTTP request 
# (Planet Analytics API base URL + desired endpoint)
collections_request_url = PAA_BASE_URL + collections_endpoint

In [ ]:
# Request Collection Items
# We're interested in the items in the collection

# Construct the URL for the subscription's Results Collection
collection_results_url = collections_request_url + subscription_id + '/' + 'items'

print("Request URL: {}".format(collection_results_url))

# Get subscription results collection
collection_items = requests.get(collection_results_url, auth=apiAuth).json()

# Pretty Print response JSON
print(json.dumps(collection_items, sort_keys=True, indent=4))

In [ ]:
# How many features do we have in this collection?

features = collection_items['features']
print('{} features in collection'.format(len(features)))

In [ ]:
# sort features by acquisition date and take latest feature

features.sort(key=lambda k: k['properties']['first_acquired'])
feature = features[-1]

print (feature)

print(feature['properties']['first_acquired'])

### Download Quad Raster

In [ ]:
feature_links = feature['links']
feature_links

In [ ]:
# Construct the URL to target quad
target_quad = list(filter(lambda link: link['rel'] == 'target-quad', feature_links))[0]['href']

print("Target (Result) Quad URL:\n{}\n".format(target_quad))

Clicking the link in the above cell will download the COG (.tiff) file!

### Visualize Roads Image

The output of the analytics road detection is a boolean image where road pixels are given a value of True and non-road pixels are given a value of False.

In [ ]:
# Set filepaths for target quad (raster), and folder where you'd like our vector files to be downloaded

filename = "TIFF_FILEPATH_HERE"
dest = "VECTOR_DESTINATION_HERE"

In [ ]:
def _open(filepath, factor=1):
    with rasterio.open(filepath) as dataset:
        height = int(dataset.height / factor)
        width = int(dataset.width / factor)
        data = dataset.read(
            out_shape=(dataset.count, height, width)
        )
    return data

def open_bool(filepath, factor=1):
    data = _open(filepath, factor=factor)
    return data[0,:,:]

def get_figsize(factor):
    return tuple(2 * [int(25/factor)])



factor = 1
figsize = (15, 15)

roads = open_bool(filepath, factor=factor)
fig = plt.figure(figsize=figsize)
# show(roads, title="footprints", cmap="binary")
show(roads[2500:3000, 0:500], title="footprints", cmap="binary")

## Convert Roads to Vector Features

### GDAL Command-Line Interface (CLI)

GDAL provides a python script that can be run via the CLI. It is quite easy to run and fast, though it doesn't allow for some of the convenient pixel-space filtering and processing that rasterio provides and we will use later on.

In [ ]:
def get_layer_name(filepath):
    # get the default layer output layer name based on the 
    # output filename. I wish there was a way to specify
    # the output layer name but attempts have failed thus far.
    return filepath.split('/')[-1].split('.')[0]

gdal_tmp_output_filename = os.path.join(dest, 'test_gdal_all.shp')
gdal_tmp_output_layer_name = get_layer_name(gdal_tmp_output_filename)
gdal_output_filename = os.path.join(dest, 'test_gdal.shp')
gdal_output_layer_name = get_layer_name(gdal_output_filename)

In [ ]:
# convert the binary image into polygons
# creates polygons for road footprints as well as regions between
# and around road footprints
!gdal_polygonize.py $filepath $gdal_tmp_output_filename

In [ ]:
# get number of features, this includes inside and outside road footprints
!ogrinfo -so  $gdal_tmp_output_filename $gdal_tmp_output_layer_name | grep 'Feature Count'

In [ ]:
# get number of road footprint features
# road footprints are associated with image value (DN) of 255
!ogrinfo -so $gdal_tmp_output_filename -sql "SELECT * FROM $gdal_tmp_output_layer_name WHERE DN=255" \
    | grep 'Feature Count'

In [ ]:
# create a new shapefile with only road footprints
!ogr2ogr -sql "SELECT * FROM $gdal_tmp_output_layer_name WHERE DN=255" \
    $gdal_output_filename $gdal_tmp_output_filename

In [ ]:
# confirm the number of road footprint features
!ogrinfo -so $gdal_output_filename -sql "SELECT * FROM $gdal_output_layer_name WHERE DN=255" \
    | grep 'Feature Count'

### Rasterio - no filtering

In this section we use rasterio to convert the binary roads raster into a vector dataset. The vectors are written to disk as a shapefile. The shapefile can be imported into geospatial programs such as QGIS or ArcGIS for visualization and further processing.

This is basic conversion to vector shapes. No filtering based on size (useful for removing small 1 or 2 pixel road segments), smoothing to remove pixel edges, or conversion to the road centerlines is performed here. These additional 'features' will be provided in sections below this one in the future.

In [ ]:
def roads_as_vectors(filepath): 
    with rasterio.open(filepath) as dataset:
        roads = dataset.read(1)
        road_mask = roads == 255 # mask non-road pixels

        # transforms roads features to image crs
        road_shapes = rfeatures.shapes(roads, mask=road_mask, connectivity=8, transform=dataset.transform)
        road_geometries = (r for r, _ in road_shapes)
        
        crs = dataset.crs
    return (road_geometries, crs)

def save_as_shapefile(output_filename, geometries, crs):
    driver='ESRI Shapefile'
    schema = {'geometry': 'Polygon', 'properties': []}
    with fiona.open(output_filename, mode='w', driver=driver, schema=schema, crs=crs) as c:
        count = 0
        for g in geometries:
            count += 1;
            c.write({'geometry': g, 'properties': {}})
        print('wrote {} geometries to {}'.format(count, output_filename))

        
road_geometries, crs = roads_as_vectors(filepath)
output_filename = os.path.join('data', 'test.shp')
save_as_shapefile(output_filename, road_geometries, crs)

### Rasterio - Filtering and Simplifying

In this section, we use `shapely` to filter the road vectors by size and simplify them so we don't have a million pixel edges.

In [ ]:
def roads_as_vectors_with_filtering(filepath, min_pixel_size=5): 
    with rasterio.open(filepath) as dataset:
        roads = dataset.read(1)
        road_mask = roads == 255 # mask non-road pixels

        # we skip transform on vectorization so we can perform filtering in pixel space
        road_shapes = rfeatures.shapes(roads, mask=road_mask, connectivity=8)
        road_geometries = (r for r, _ in road_shapes)
        geo_shapes = (sshape(g) for g in road_geometries)

        # filter to shapes bigger than min_pixel_size
        geo_shapes = (s for s in geo_shapes if s.area > min_pixel_size)
    
        # simplify so we don't have a million pixel edge points
        tolerance = 1 #1.5
        geo_shapes = (g.simplify(tolerance, preserve_topology=False)
                      for g in geo_shapes)

        # apply image transform    
        # rasterio transform: (a, b, c, d, e, f, 0, 0, 1), c and f are offsets
        # shapely: a b d e c/xoff f/yoff
        d = dataset.transform
        shapely_transform = [d[0], d[1], d[3], d[4], d[2], d[5]]
        proj_shapes = (shapely.affinity.affine_transform(g, shapely_transform)
                       for g in geo_shapes)
        
        road_geometries = (shapely.geometry.mapping(s) for s in proj_shapes)
        
        crs = dataset.crs
    return (road_geometries, crs)

In [ ]:
road_geometries_filt, crs = roads_as_vectors_with_filtering(filepath)
output_filename = os.path.join('data', 'test_filt.shp')
save_as_shapefile(output_filename, road_geometries_filt, crs)